# Chess Engine with PyTorch

## Imports

In [6]:
pip install python-chess tqdm numpy torch

  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 16.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.4 MB/s eta 0:00:00a 0:00:01
Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 14.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import numpy as np # type: ignore
import time
import torch
import torch.nn as nn # type: ignore
import torch.optim as optim # type: ignore
from torch.utils.data import DataLoader # type: ignore
from chess import pgn # type: ignore
from tqdm import tqdm # type: ignore

# Data preprocessing

## Load data

In [8]:
def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

files = [file for file in os.listdir("./") if file.endswith(".pgn")]
LIMIT_OF_FILES = min(len(files), 28)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"./{file}"))
    if i >= LIMIT_OF_FILES:
        break
    i += 1

  0%|          | 0/1 [01:18<?, ?it/s]


In [9]:
print(f"GAMES PARSED: {len(games)}")

GAMES PARSED: 48932


## Convert data into tensors

In [11]:
from auxiliary_func import create_input_for_nn, encode_moves

In [12]:
X, y = create_input_for_nn(games)

print(f"NUMBER OF SAMPLES: {len(y)}")

NUMBER OF SAMPLES: 4175888


In [13]:
X = X[0:2500000]
y = y[0:2500000]

In [14]:
y, move_to_int = encode_moves(y)
num_classes = len(move_to_int)

In [15]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Preliminary actions

In [16]:
from dataset import ChessDataset
from model import ChessModel

In [17]:
# Create Dataset and DataLoader
dataset = ChessDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# Model Initialization
model = ChessModel(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Using device: cpu


# Training

In [18]:
num_epochs = 50
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(dataloader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU
        optimizer.zero_grad()

        outputs = model(inputs)  # Raw logits

        # Compute loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        running_loss += loss.item()
    end_time = time.time()
    epoch_time = end_time - start_time
    minutes: int = int(epoch_time // 60)
    seconds: int = int(epoch_time) - minutes * 60
    print(f'Epoch {epoch + 1 + 50}/{num_epochs + 1 + 50}, Loss: {running_loss / len(dataloader):.4f}, Time: {minutes}m{seconds}s')

100%|██████████| 39063/39063 [16:18<00:00, 39.91it/s]  


Epoch 51/101, Loss: 3.3098, Time: 16m18s


100%|██████████| 39063/39063 [17:32<00:00, 37.13it/s]  


Epoch 52/101, Loss: 2.3576, Time: 17m32s


100%|██████████| 39063/39063 [18:14<00:00, 35.68it/s]  


...

Epoch 59/101, Loss: 1.5809, Time: 15m51s


100%|██████████| 39063/39063 [17:41<00:00, 36.80it/s]


Epoch 60/101, Loss: 1.5366, Time: 17m41s


 67%|██████▋   | 26359/39063 [11:38<05:36, 37.73it/s] 


KeyboardInterrupt: 

# Save the model and mapping

In [18]:
# Save the model
torch.save(model.state_dict(), "../../models/TORCH_100EPOCHS.pth")

In [16]:
import pickle

with open("../../models/heavy_move_to_int", "wb") as file:
    pickle.dump(move_to_int, file)